In [1]:
#%run Vid_Processing.ipynb

In [2]:
import import_ipynb
from Functions import *

importing Jupyter notebook from Functions.ipynb


cap = open_vid('VDB\V.mp4')
Frames = get_frames(cap)
print("FPS: ",cap.get(cv2.CAP_PROP_FPS))
F = Frames.copy()
loc=[[150,50]]
sizes = [[50,100]]
N = 15
F[N] = occlusions(F[N],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+1] = occlusions(F[N+1],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+2] = occlusions(F[N+2],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
_ = OFM(F)
_ = OFV(F,15)
_ = OFLK(F,15)

file = 'IDB/coast.jpg'
img = cv2.imread(file)
toon = cartoonize_image(img,10,10)
toon1 = cartoonize_image(img,10,10)
cv2.imshow('toon',toon)
cv2.imshow('toon1',toon1)
cv2.waitKey(0)
cv2.destroyAllWindows()

cap = open_vid('VDB/U.mp4')
Frames = get_frames(cap)
fps = cap.get(cv2.CAP_PROP_FPS)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
Toon = [cv2.resize(cartoonize_image(cv2.resize(f,(600,600))),(w,h)) for f in Frames]
save_vid(Toon,'Cartoonized/U_toonzz.mp4',fps)

cap = open_vid('VDB/U.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
Frames = get_frames(cap)
cap = open_vid('Cartoonized/U_toon.mp4')
Toon = get_frames(cap)
play_frames(Frames,fps)
play_frames(Toon,fps)
_ = OFV(Frames,20)
_ = OFV(Toon,20)

img = cv2.imread('IDB/bunny.jpg')
display_frame(img)
display_frame(cartoonize_image(cv2.resize(img,(500,500))))

In [8]:
cap = open_vid('VDB\M.mp4')
_,_,fps,_ = get_props(cap)
Frames = get_frames(cap)

Width:  640
Height:  296
FPS:  30.0
Frame Count:  346


In [13]:
_ = OFV(Frames[100:200],20)

In [5]:
#Explicar con texto
#Cambiar en cada escena
#Indices de variabilidad
#Reconocer cosas en imagen
#Escenas con poca variabilidad
#Cambiar kernels y parametros
#Entorno/Interfaz